W celu wyznaczenia drogi przez labirynt zbudowano algorytm genetyczny z zastosowaniem biblioteki pygad. Do niktórych obliczeń po drodze potrzebna była również biblioteka numpy, natomiast pomiar czasu działania algorytmu był możliwy dzięki bibliotece time.

In [35]:
import numpy as np
import pygad
import time

Labirynt zakodowano za pomocą 0 i 1. Zero oznacza pole dozwolone, po którym można się poruszać, natomiast pole czarne oznacza ścianę; na te pola nie wolno wchodzić. Włącznie ze ścianami na granicy labiryntu, ma on wymiary 12x12, zatem współrzędne w python będą numerowane od 0 do 11. Pole startu znajduje się na polu (1,1), a meta na polu (10,10).

In [36]:
labyrinth = np.array([
            [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
            [1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1],
            [1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1],
            [1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1],
            [1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1],
            [1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1],
            [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1],
            [1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1],
            [1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1],
            [1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1],
            [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
            [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
        ])

Ruchy przez labirynt zakodowano w następujący sposób:
0 - bez ruchu;
1 - w lewo;
2 - w prawo;
3 - w górę;
4 - w dół;

Funkcja fitnessu oceniała trasy o długości 30, kodowane za pomocą powyższych genów (0,...,4). Karała za wejście w ścianę oraz stanie w miejscu, dopóki nie dotarto do mety. Ponadto ignorowała ruchy przez ściany. Po dojściu do mety przestawała karać za brak ruchu - zamiast tego nagradzała za pozostanie w mecie. Wartości funkcji fitnessu były ustandaryzowaną sumą z wagami: metryki Manhattan odległości od mety z wagą 2, kar za "wejścia" na ściany i powtórzenie pozycji przed osiągnięciem mety z wagami 1 oraz wreszcie bonusu za pozostanie w mecie z wagą 2. Ponieważ maksymalna wartość tej sumy wynosiła 56, to w celu standaryzacji podzielono tę sumę rpzez 56.

In [37]:
def fitness_fun(route, route_idx=None):

    # Zaczynamy w (1, 1)
    i, j = 1, 1
    position = [0]
    is_problem = 0
    zero_no = 0
    bonus = 0

    for move in route:  # Zmieniamy położenie w zależności od wykonanego ruchu

        if position[-1] == [10, 10] and move == 0:  # bonus za pozostanie w mecie
            bonus += 1
            continue

        if move == 1:
            new_j, new_i = j - 1, i
        elif move == 2:
            new_j, new_i = j + 1, i
        elif move == 3:
            new_i, new_j = i - 1, j
        elif move == 4:
            new_i, new_j = i + 1, j
        else:
            zero_no += 1
            new_i, new_j = i, j

        if 0 <= new_i <= 11 and 0 <= new_j <= 11:
            if labyrinth[new_i, new_j] == 0:
                i, j = new_i, new_j
                position.append([i, j])
                if position.count([i, j]) > 1:  # kara za powtórzenie pozycji
                    is_problem += 1
            else:
                is_problem += 1.25  # kara za zmarnowanie ruchu na odbicie się od ściany

    fitness = ((20 - abs(10 - i) - abs(10 - j)) * 2 - is_problem - zero_no + bonus * 2) / 56

    return fitness

Ponieważ koszt obliczeniowy algorytmu genetycznego rośnie rpzede wszysktim z rozmiarem populacji, a nie liczbą generacji, to mimo złożonego problemu nie generowano dużych populacji. Dano za to stosunkowo dużo czasu na wykształcenie się preferowanych chromosomów.

In [38]:
gene_space = [0, 1, 2, 3, 4]
num_generations = 3000
sol_per_pop = 500
num_parents_mating = 250
elite_size = 20
num_genes = 30

In [39]:
ga_instance = pygad.GA(
    gene_space=gene_space,
    num_genes=num_genes,
    num_generations=num_generations,
    num_parents_mating=num_parents_mating,
    fitness_func=fitness_fun,
    sol_per_pop=sol_per_pop,
    keep_parents=elite_size,
    parent_selection_type="rank",
    mutation_type="random",
    mutation_probability=0.25,
    stop_criteria="reach_1"
)

Algorytm wykonano 10 razy, za każdym razem zapamiętując czas wykonywania, nr generacji najlepszego rozwiązania, wartość funkcji fitnessu tego rozwiązania oraz oczywiście samo to rozwiązanie.

In [40]:
fitness_list = []
times = []
output_list = []
generations_no = []

In [41]:
for i in range(10):
    # Zaczynam pomiar czasu
    start = time.time()

    # uruchomienie algorytmu
    ga_instance.run()

    # Kończę pomiar czasu
    end = time.time()
    times.append(end - start)

    # Parametry danego najlepszego rozwiązania
    solution, solution_fitness, solution_idx = ga_instance.best_solution()
    generations_no.append(ga_instance.best_solution_generation)

    fitness_list.append(solution_fitness)
    output_list.append(solution)

Następnie wypisano historie wyników i średnie z ich parametrów.

In [42]:
print("Średni czas działania algorytmu genetycznego: {}".format(np.mean(times)))
print("Średnia wartość funkcji fitnessu najlepszego rozwiązania: {}".format(np.mean(fitness_list)))
print("Średnia liczba generacji do najlepszego rozwiązania: {}".format(np.mean(generations_no)))

print("Historia wyników :")
for i in range(len(output_list)):
    print(output_list[i])

Średni czas działania algorytmu genetycznego: 124.35253026485444
Średnia wartość funkcji fitnessu najlepszego rozwiązania: 0.8808035714285714
Średnia liczba generacji do najlepszego rozwiązania: 674.7
Historia wyników :
[2. 2. 4. 2. 2. 3. 2. 2. 3. 4. 4. 4. 1. 4. 2. 4. 2. 2. 4. 1. 2. 4. 4. 4.
 1. 2. 0. 0. 0. 0.]
[2. 2. 4. 2. 2. 3. 3. 2. 2. 4. 4. 4. 1. 4. 2. 4. 2. 2. 4. 2. 2. 4. 4. 4.
 0. 2. 0. 0. 0. 0.]
[2. 2. 4. 2. 2. 3. 3. 2. 2. 4. 4. 4. 1. 4. 2. 4. 2. 2. 4. 4. 2. 4. 4. 4.
 0. 0. 0. 0. 0. 0.]
[2. 2. 4. 2. 2. 3. 1. 2. 2. 4. 4. 4. 1. 4. 2. 4. 2. 2. 4. 2. 3. 4. 4. 4.
 0. 0. 0. 0. 0. 0.]
[2. 2. 4. 2. 2. 3. 3. 2. 2. 4. 4. 4. 1. 4. 2. 4. 2. 2. 4. 2. 2. 4. 4. 4.
 0. 0. 0. 0. 0. 0.]
[2. 2. 4. 2. 2. 3. 1. 2. 2. 4. 4. 4. 1. 4. 2. 4. 2. 2. 4. 2. 2. 4. 4. 4.
 0. 0. 0. 0. 0. 0.]
[2. 2. 4. 2. 2. 3. 1. 2. 2. 4. 4. 4. 1. 4. 2. 4. 2. 2. 4. 2. 4. 4. 4. 4.
 0. 0. 0. 0. 0. 0.]
[2. 2. 4. 2. 2. 3. 1. 2. 2. 4. 4. 4. 1. 4. 2. 4. 2. 2. 4. 4. 2. 4. 4. 4.
 0. 0. 0. 0. 0. 0.]
[2. 2. 4. 2. 2. 3. 1. 2. 2. 4. 4. 4.

In [43]:
for index in range(len(output_list)):
    print(output_list[index])

    # Zaczynamy w (1, 1)
    i, j = 1, 1
    position = [0]
    lab_with_route = np.array([  # będę nadpisywać każdorazowo pola, na których był ruch
            [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
            [1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1],
            [1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1],
            [1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1],
            [1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1],
            [1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1],
            [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1],
            [1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1],
            [1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1],
            [1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1],
            [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
            [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
        ])

    for move in output_list[index]:  # analogicznie do funkcji fitnessu sprawdzamy kolejne położenia
        if position[-1] == [10, 10] and move == 0:
            continue

        if move == 1:
            new_j, new_i = j - 1, i
        elif move == 2:
            new_j, new_i = j + 1, i
        elif move == 3:
            new_i, new_j = i - 1, j
        elif move == 4:
            new_i, new_j = i + 1, j
        else:
            new_i, new_j = i, j

        if 0 <= new_i <= 11 and 0 <= new_j <= 11:
            if lab_with_route[new_i, new_j] != 1:
                i, j = new_i, new_j
                position.append([i, j])
                lab_with_route[i, j] = 2

    for index1 in range(12):
        line = ""
        for index2 in range(12):
            if lab_with_route[index1, index2] == 0:
                line += ' '
            elif lab_with_route[index1, index2] == 1:
                line += '#'
            else:
                line += 'X'
        print(line)
    
    # print(lab_with_route)

[2. 2. 4. 2. 2. 3. 2. 2. 3. 4. 4. 4. 1. 4. 2. 4. 2. 2. 4. 1. 2. 4. 4. 4.
 1. 2. 0. 0. 0. 0.]
############
# XX#XXX#  #
###XXX#X## #
#   # #X   #
# # ##XX## #
#  ## XX#  #
#     #XXX##
# #  ## #XX#
# ###   ##X#
# # ## # #X#
# #      XX#
############
[2. 2. 4. 2. 2. 3. 3. 2. 2. 4. 4. 4. 1. 4. 2. 4. 2. 2. 4. 2. 2. 4. 4. 4.
 0. 2. 0. 0. 0. 0.]
############
# XX#XXX#  #
###XXX#X## #
#   # #X   #
# # ##XX## #
#  ## XX#  #
#     #XXX##
# #  ## #XX#
# ###   ##X#
# # ## # #X#
# #       X#
############
[2. 2. 4. 2. 2. 3. 3. 2. 2. 4. 4. 4. 1. 4. 2. 4. 2. 2. 4. 4. 2. 4. 4. 4.
 0. 0. 0. 0. 0. 0.]
############
# XX#XXX#  #
###XXX#X## #
#   # #X   #
# # ##XX## #
#  ## XX#  #
#     #XXX##
# #  ## #XX#
# ###   ##X#
# # ## # #X#
# #       X#
############
[2. 2. 4. 2. 2. 3. 1. 2. 2. 4. 4. 4. 1. 4. 2. 4. 2. 2. 4. 2. 3. 4. 4. 4.
 0. 0. 0. 0. 0. 0.]
############
# XX#XXX#  #
###XXX#X## #
#   # #X   #
# # ##XX## #
#  ## XX#  #
#     #XXX##
# #  ## #XX#
# ###   ##X#
# # ## # #X#
# #       X#
############
[2. 